In [1]:
import keras
from keras import layers
from keras import ops
from keras.layers import TextVectorization
import numpy as np
import pandas as pd
import os
import string
import random
import tensorflow as tf
import tensorflow.data as tf_data
import tensorflow.strings as tf_strings

os.environ["KERAS_BACKEND"] = "tensorflow"

2024-03-11 18:03:43.681516: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-11 18:03:43.681898: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-11 18:03:43.684647: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-11 18:03:43.718922: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-11 18:03:44.476837: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [2]:
df = pd.read_csv('midi_df_full.csv', index_col=0, escapechar='|')

In [3]:
# sample list
df_exploration = df.sample(100)

In [4]:
# df_exploration['tokenized_guitar'] = df_exploration['tokenized_guitar'].apply(lambda x: x[1:-1].split(', '))

In [5]:
# df_exploration['tokenized_drums'] = df_exploration['tokenized_drums'].apply(lambda x: x[1:-1].split(', '))

In [6]:
# df_exploration['tokenized_guitar'] = df_exploration['tokenized_guitar'].apply(lambda x: [int(i) for i in x])

In [7]:
# df_exploration['tokenized_drums'] = df_exploration['tokenized_drums'].apply(lambda x: [int(i) for i in x])

In [8]:
# df['tokenized_guitar'] = df['tokenized_guitar'].apply(lambda x: x.replace('[UNK]', str(0)))

In [9]:
# df['tokenized_drums'] = df['tokenized_drums'].apply(lambda x: x.replace('[UNK]', str(0)))

In [10]:
df_exploration.reset_index(drop=True, inplace=True)

In [11]:
max(df_exploration['tokenized_guitar'].apply(lambda x : len(x)))

2015

In [12]:
df_exploration.shape

(100, 7)

In [13]:
train_data = df_exploration.iloc[:89].loc[:,['tokenized_guitar', 'tokenized_drums']]
test_data = df_exploration.iloc[89:].loc[:,['tokenized_guitar', 'tokenized_drums']]

In [16]:
MAX_TOKENS=256
def prepare_batch(guitar, drums):
    guitar = guitar[:, :MAX_TOKENS]    # Trim to MAX_TOKENS.
    guitar = guitar.to_tensor()  # Convert to 0-padded dense Tensor

    drums = drums[:, :(MAX_TOKENS+1)]
    drums_original = drums[:, :-1].to_tensor()
    drums_shifted = drums[:, 1:].to_tensor()

    return (guitar, drums_original), drums_shifted

In [17]:
BUFFER_SIZE = 2000
BATCH_SIZE = 32

In [18]:
def make_batches(ds):
  return (
      ds
      .shuffle(BUFFER_SIZE)
      .batch(BATCH_SIZE)
      .map(prepare_batch, tf.data.AUTOTUNE)
      .prefetch(buffer_size=tf.data.AUTOTUNE))

In [19]:
ds_train = tf.data.Dataset.from_tensor_slices((train_data['tokenized_guitar'].values, train_data['tokenized_drums'].values))    
ds_test = tf.data.Dataset.from_tensor_slices((test_data['tokenized_guitar'].values, test_data['tokenized_drums'].values))    

In [20]:
def convert_to_ragged(guitar, drums):
    guitar = tf.strings.regex_replace(guitar, "[\[\]']", "")
    drums = tf.strings.regex_replace(drums, "[\[\]']", "")

    split_guitar = tf.strings.split(guitar, ', ')
    split_drums = tf.strings.split(drums, ', ')
    
    ragged_tensor_guitar = tf.RaggedTensor.from_row_lengths(split_guitar, row_lengths=tf.strings.length(split_guitar))
    ragged_tensor_drums = tf.RaggedTensor.from_row_lengths(split_drums, row_lengths=tf.strings.length(split_drums))
    return ragged_tensor_guitar, ragged_tensor_drums

In [21]:
ds_train = ds_train.map(convert_to_ragged)
ds_test =  ds_test.map(convert_to_ragged)

In [22]:
# Create training and validation set batches.
train_batches = make_batches(ds_train)
val_batches = make_batches(ds_test)

In [23]:
dd=train_batches.take(2)

In [24]:
for (guitar, drums), drums_shifted in dd:
    print(guitar)

2024-03-11 18:10:29.564790: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: assertion failed: [Arguments to _from_row_partition do not form a valid RaggedTensor] [Condition x == y did not hold element-wise:] [x (RaggedFromRowLengths/strided_slice_1:0) = ] [754] [y (RaggedFromRowLengths/strided_slice:0) = ] [168]
	 [[{{node RaggedFromRowLengths/assert_equal_1/Assert/Assert}}]]


InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_3_device_/job:localhost/replica:0/task:0/device:CPU:0}} assertion failed: [Arguments to _from_row_partition do not form a valid RaggedTensor] [Condition x == y did not hold element-wise:] [x (RaggedFromRowLengths/strided_slice_1:0) = ] [754] [y (RaggedFromRowLengths/strided_slice:0) = ] [168]
	 [[{{node RaggedFromRowLengths/assert_equal_1/Assert/Assert}}]] [Op:IteratorGetNext] name: 